In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

###LOAD IN THE SHEET
dataframe = pd.read_excel('../data/synthetic_participant_dose_data.xlsx',sheet_name=None)

###GET THE NAMES OF ALL OF THE POTENTIAL SHEETS
sheet_names = ", ".join(dataframe.keys())
print(f"Sheet names = {sheet_names}")

###Get the sheets you want
participants = pd.read_excel('../data/synthetic_participant_dose_data.xlsx',sheet_name='participants')
doses = pd.read_excel('../data/synthetic_participant_dose_data.xlsx',sheet_name='doses')

###check for missing values and remove those
#print(doses.isna().any())